In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

## Basic app to extract from a ChatMessage the song and artist a user wants to play

**Define your extraction goal (called "the response schema")**

In [2]:
from langchain.output_parsers import ResponseSchema

In [3]:
response_schemas = [
    ResponseSchema(
        name="singer",
        description="name of the singer"
    ),
    ResponseSchema(
        name="song",
        description="name of the song"
    )
]

**Create the Output Parser that will extract the data**

In [4]:
from langchain.output_parsers import StructuredOutputParser

In [5]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

**Create the format instructions**

In [6]:
format_instructions = output_parser.get_format_instructions()

In [7]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"singer": string  // name of the singer
	"song": string  // name of the song
}
```


**Create the ChatPromptTemplate**

In [11]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import HumanMessagePromptTemplate

In [10]:
chat_model = ChatOpenAI()

In [22]:
template = """
Given a command from the user,
extract the artist and song names
{format_instructions}
{user_prompt}
"""

In [24]:
prompt = ChatPromptTemplate(
    messages=[
        HumanMessagePromptTemplate.from_template(template)
    ],
    input_variables={"user_prompt"},
    partial_variables={"format_instructions": format_instructions}
)

**Enter the chat message from the user**

In [25]:
user_message = prompt.format_prompt(
    user_prompt="I like the song New York, New York by Frank Sinatra"
)

In [27]:
user_chat_message = chat_model(user_message.to_messages())

**Extract singer and song from the user message**

In [29]:
extraction = output_parser.parse(user_chat_message.content)

In [30]:
print(extraction)

{'singer': 'Frank Sinatra', 'song': 'New York, New York'}


In [31]:
print(type(extraction))

<class 'dict'>
